In [ ]:
from pathlib import Path
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('dark_background')


In [ ]:
SAVE_IMAGES = True

def plot(axes, title=None, file=None, ylim=None, pppp=plt, xticks=None):
    if title: axes.set_title(title)
    axes.legend(loc="upper left")
    axes.set_xlim(None, None)
    axes.autoscale()
    pppp.tight_layout()
    if ylim: 
        if type(ylim) == int: pppp.ylim(ylim)
        else: pppp.ylim(ylim[0], ylim[1])
    # print(f"ylim={ylim} type={type(ylim)} int={type(ylim)==int}")
    if xticks: pppp.xticks(xticks, rotation=90)
    if SAVE_IMAGES and file: pppp.savefig(file)
    return pppp


In [ ]:
data_vacinas = pd.read_csv(Path.cwd() / '..' / '..' /'vacinas.csv')
data_vacinas_detalhes = pd.read_csv(Path.cwd() / '..' / '..' /'vacinas_detalhe.csv')


In [ ]:
pop_cols = [col for col in data_vacinas_detalhes.columns if 'populacao1' in col]
data_vacinas_detalhes[pop_cols].max().transpose()


In [ ]:
df = pd.merge(data_vacinas, data_vacinas_detalhes, how='right', on='data', suffixes=("_diario", "_semanal"))
df['data'] = df['data'].apply(lambda x: f"{x[6:]}-{x[3:5]}-{x[0:2]}")
df.set_index("data", inplace=True)

for k in ['', '1', '2']:
    df[f'doses{k}_diff'] = df[f'doses{k}_diario'] - df[f'doses{k}_continente']
    #df[f'doses{k}_diff'] = df[f'doses{k}_diario'] - df[f'doses{k}_semanal']
df['doses12_diff'] = df['doses1_diff'] + df['doses2_diff']
df[ [c for c in df.columns if '_diff' in c] ]


In [ ]:
df = pd.merge(data_vacinas, data_vacinas_detalhes, how='left', on='data', suffixes=("_diario", "_semanal"))
df.ffill(inplace=True)
df['data'] = df['data'].apply(lambda x: f"{x[6:]}-{x[3:5]}-{x[0:2]}")
df.set_index("data", inplace=True)


In [ ]:
(df['doses_diario'] - df['doses_continente']).tail(30)


In [ ]:
# FOO

In [ ]:
# ",".join(data_vacinas_detalhes.columns)


In [ ]:
# negativos

# parcialmente_novas é normal ser negativo - basta haver mais doses2 que doses1
df.drop("pessoas_vacinadas_parcialmente_novas", axis=1, inplace=True)

# 29-03-2021	pessoas_vacinadas_completamente_novas -640.0
#df[ (df < 0).any(axis=1) ]
df[ (df.index >= "2021-03-20") & (df.index <= "2021-03-30") ][ [col for col in df.columns if not('perc' in col or 'popu' in col) and ('diario' in col or 'continente' in col or col in ['doses1_semanal', 'doses1_novas_semanal', 'doses2_semanal', 'doses2_novas_semanal'] )]]


In [ ]:
data_vacinas_detalhes[ [col for col in data_vacinas_detalhes.columns if 'novas' not in col and 'desconhecido' not in col and 'unk' not in col] ].tail(3)


In [ ]:
df_vacinas = data_vacinas.copy()
df_vacinas.drop( [x for x in df_vacinas.columns if '_novas' in x or 'pessoas' in x], axis=1, inplace=True)
#df_vacinas.columns

df_vacinas_detalhes = data_vacinas_detalhes.copy()
df_vacinas_detalhes.drop( [x for x in df_vacinas_detalhes.columns if '_novas' in x or 'pessoas' in x], axis=1, inplace=True)
#df_vacinas_detalhes.columns

#df = df_vacinas_detalhes.merge(df_vacinas, how='left', on='data', suffixes=("_semanal", "_diario"))
df = df_vacinas.merge(df_vacinas_detalhes, how='left', on='data', suffixes=("_diario", "_semanal"))
df.ffill(inplace=True)
#",".join(df.columns)

for i in ['', '1', '2']:
    df[f'doses{i}_diff'] = df[f'doses{i}_continente'] - df[f'doses{i}_diario']
cols = ["data"] + [ f'doses{i}_{j}' for i in ['', '1', '2'] for j in ['diff', 'continente', 'diario', 'semanal']]

df[cols].tail(10)
#df[df.data == '28-03-2021']


In [ ]:
df = df_vacinas_detalhes.merge(df_vacinas, how='left', on='data', suffixes=("_semanal", "_diario"))

for i in ['', '1', '2']:
    df[f'doses{i}_diff'] = df[f'doses{i}_continente'] - df[f'doses{i}_diario']
cols = ["data"] + [ f'doses{i}_{j}' for i in ['', '1', '2'] for j in ['diff', 'continente', 'diario', 'semanal']]

df[cols].tail(10)



In [ ]:
df = df_vacinas_detalhes.merge(df_vacinas, how='left', on='data', suffixes=("_semanal", "_diario"))
for k in ['','1','2']:
    df[f'doses{k}_diff'] = df[f'doses{k}_continente'] - df[f'doses{k}_diario']
    df[f'doses{k}_diff_novas'] = df[f'doses{k}_diff'].diff(1)
df = df[17:] # 03-05-2021
#df[['data', 'doses_diff', 'doses_diff_novas', 'doses2_diff', 'doses2_diff_novas', 'doses1_diff', 'doses1_diff_novas']]

df[['data', 'doses_diff_novas']]

In [ ]:
df = df_vacinas_detalhes.copy()
for i in [1,2]: 
    df[f'doses{i}_diff'] = df[f'doses{i}'] - df[f'doses{i}_continente']
    df[f'doses{i}_diff_novas'] = df[f'doses{i}_diff'].diff(1)

df = df[1:]
df[[
    'data',
    'doses2_diff', 'doses2_diff_novas', 'doses2_madeira', 'doses2_açores',
    'doses1_diff', 'doses1_diff_novas', 'doses1_madeira', 'doses1_açores',
    ]]



In [ ]:
# ",".join(list(df.columns))


In [ ]:
# idades
df[ [col for col in df.columns if 'perc' in col][18:] ].tail(10)


In [ ]:
# ars
df[ [col for col in df.columns if 'perc' in col][:18] ].tail(10)


In [ ]:
# ars
dfl = df[ [col for col in df.columns if 'perc' in col][:18] ][-1:]

doses1, doses2 = float(dfl['doses1_perc']), float(dfl['doses2_perc'])
for region in ['arsnorte', 'arscentro', 'arslvt', 'arsalentejo', 'arsalgarve', 'madeira', 'açores']:
    d1, d2 = float(dfl[f'doses1_perc_{region}']), float(dfl[f'doses2_perc_{region}'])
    print(f"region={region} vacinados: {round( (d2-doses2)*100, 1)}% 1ª dose: {round((d1-doses1)*100, 1)}% ")



In [ ]:

df = data_vacinas[ [col for col in data_vacinas.columns if 'novas' not in col] ].copy()
df[ df['doses'] < df['pessoas_vacinadas_parcialmente'] ]




In [ ]:
df_vacinas_detalhes[ df_vacinas_detalhes['data'] == '11-01-2021' ][ ['data', 'doses1_madeira', 'doses2_madeira', 'doses1_açores', 'doses2_açores', 'doses1', 'doses2', 'doses1_continente', 'doses2_continente']]


In [ ]:
print(83682.0 - 75581  + 75280.0)


In [ ]:
df_vacinas = data_vacinas.copy()
df_vacinas.drop( [x for x in df_vacinas.columns if '_novas' in x ], axis=1, inplace=True)

df_vacinas['foo'] = df_vacinas['doses'] - df_vacinas['doses1'] - df_vacinas['doses2']
df_vacinas['bar'] = df_vacinas['pessoas_vacinadas_completamente'] + df_vacinas['pessoas_vacinadas_parcialmente'] + df_vacinas['pessoas_vacinadas_parcialmente']
df_vacinas[df_vacinas['foo'] != 0]


In [ ]:
df_vacinas_detalhes

In [ ]:
data_vacinas2 = pd.read_csv(Path.cwd() / '..' / '..' /'vacinas.csv', parse_dates=['data'], index_col='data', dayfirst=True)
data_vacinas_detalhe2 = pd.read_csv(Path.cwd() / '..' / '..' /'vacinas_detalhe.csv', parse_dates=['data'], index_col='data', dayfirst=True)
df_vacinas_detalhes2 = data_vacinas_detalhe2

dfr = df_vacinas_detalhes2.recebidas.diff(1)

#axes = dfr.plot(figsize=(15,5))
fig, axes = plt.subplots(figsize=(15,5))
axes.plot(dfr, label="recebidas")


axes.legend(loc="upper left")
plt.xticks(list(reversed(list(reversed(list(set(list(dfr.index))))))), rotation=90)
plt.tight_layout()
#if SAVE_IMAGES: plt.savefig(f"../../temp/vacinas_ecdc_previsao_accumulado.png")
plt.show()


In [ ]:
# Pessoas com pelo menos 1 dose

POP_IDADE = {
    '0_9':     433332 + 461299,  #  0-04 + 05-09
    '10_19':   507646 + 549033,  # 10-14 + 15-19
    '20_29':   544575 + 547505,  # 20-24 + 25-29
    '30_39':   571355 + 679093,  # 30-34 + 35-39
    '40_49':   792670 + 782555,  # 40-44 + 45-49
    '50_59':   747581 + 734540,  # 50-54 + 55-59
    '60_69':   672758 + 620543,  # 60-64 + 65-69
    '70_79':   544016 + 429107,  # 70-74 + 75-79
    '80_plus': 352218 + 316442,  # 80-84 + 85 ou mais
}

IDADES = {
  # total 10_298_253
  '0_17': 1_701_687 - POP_IDADE['0_9'],
  # adultos 8_596_566
  # 70% 6_017_596
  '18_24': 781_176,
  '25_49': 3_331_561,
  '50_64': 2_174_181,
  '65_79': 1_628_058,
  '80+': 681_590,
}

def p(save=None):
    plt.xticks(list(reversed(list(reversed(list(set(list(dfr.index))))))), rotation=90)
    plt.tight_layout()
    #plt.ylim(0, 100)
    if SAVE_IMAGES and save: plt.savefig(f"../../temp/{save}.png")
    plt.show()



In [ ]:

fig, axes = plt.subplots(figsize=(15,5))
plt.title("Pessoas vacinação iniciada, semanal")
for idade in ['0_17', '18_24', '25_49', '50_64', '65_79', '80+']:
    dfr = df_vacinas_detalhes2[f'pessoas_inoculadas_{idade}'].diff(1)
    axes.plot(dfr, label=f"{idade}", lw=2, marker='o')
    axes.legend(loc="upper left")
p(save='vacinas_iniciada_semanal')

for idade in ['0_17', '18_24', '25_49', '50_64', '65_79', '80+']:
    fig, axes = plt.subplots(figsize=(15,5))
    plt.title(f"Pessoas vacinação iniciada, semanal, idade {idade}")
    dfr = df_vacinas_detalhes2[f'pessoas_inoculadas_{idade}'].diff(1)
    axes.plot(dfr, label=f"{idade}", lw=2, marker='o')
    axes.legend(loc="upper left")
    p(save=f'vacinas_iniciada_semanal_{idade}')


In [ ]:

fig, axes = plt.subplots(figsize=(15,5))
plt.title("Pessoas vacinação iniciadda, acumulado, percentagem")
for idade in ['0_17', '18_24', '25_49', '50_64', '65_79', '80+']:
    #dfr = df_vacinas_detalhes2[f'pessoas_inoculadas_{idade}'] #.diff(1)
    dfr = df_vacinas_detalhes2[f'pessoas_inoculadas_{idade}'] * 100 / IDADES[idade]
    axes.plot(dfr, label=f"{idade}", lw=2, marker='o')
    axes.legend(loc="upper left")
plt.ylim(0, 100*1.01)
p(save='vacinas_iniciada_acumulado_percentagem')

for idade in ['0_17', '18_24', '25_49', '50_64', '65_79', '80+']:
    fig, axes = plt.subplots(figsize=(15,5))
    plt.title(f"Pessoas vacinação iniciadda, acumulado, percentagem, idade {idade}")
    #dfr = df_vacinas_detalhes2[f'pessoas_inoculadas_{idade}'] #.diff(1)
    dfr = df_vacinas_detalhes2[f'pessoas_inoculadas_{idade}'] * 100 / IDADES[idade]
    axes.plot(dfr, label=f"{idade}", lw=2, marker='o')
    axes.legend(loc="upper left")
    plt.ylim(0, 100*1.01)
    p(save=f'vacinas_iniciada_acumulado_percentagem_{idade}')


In [ ]:

fig, axes = plt.subplots(figsize=(15,5))
plt.title("Pessoas pelo menos uma dose, acumulado")
for idade in ['0_17', '18_24', '25_49', '50_64', '65_79', '80+']:
    dfr = df_vacinas_detalhes2[f'pessoas_inoculadas_{idade}'] #.diff(1)
    #dfr = df_vacinas_detalhes2[f'pessoas_inoculadas_{idade}'] * 100 / IDADES[idade]
    axes.plot(dfr, label=f"{idade}", lw=2, marker='o')
    axes.axhline(y=IDADES[idade], alpha=0.5, ls='--', color=plt.gca().lines[-1].get_color() )
    axes.legend(loc="upper left")
p(save='vacinas_iniciada_acumulado')

for idade in ['0_17', '18_24', '25_49', '50_64', '65_79', '80+']:
    fig, axes = plt.subplots(figsize=(15,5))
    plt.title(f"Pessoas pelo menos uma dose, acumulado, idade {idade}, população {IDADES[idade]}")
    dfr = df_vacinas_detalhes2[f'pessoas_inoculadas_{idade}'] #.diff(1)
    #dfr = df_vacinas_detalhes2[f'pessoas_inoculadas_{idade}'] * 100 / IDADES[idade]
    axes.plot(dfr, label=f"{idade}", lw=2, marker='o')
    axes.axhline(y=IDADES[idade], alpha=0.5, ls='--', color=plt.gca().lines[-1].get_color() )
    axes.legend(loc="upper left")
    plt.ylim(0, IDADES[idade]*1.05)
    p(save=f'vacinas_iniciada_acumulado_{idade}')


In [ ]:
#data_vacinas2 = pd.read_csv(Path.cwd() / '..' / '..' /'vacinas.csv', parse_dates=['data'], index_col='data', dayfirst=True)
#data_vacinas_detalhes2 = pd.read_csv(Path.cwd() / '..' / '..' /'vacinas_detalhe.csv', parse_dates=['data'], index_col='data', dayfirst=True)
#df_vacinas = data_vacinas2.copy()
#df_vacinas_detalhes = data_vacinas_detalhes2.copy()

df_vacinas = data_vacinas.copy()
df_vacinas_detalhes = data_vacinas_detalhes.copy()

df_vacinas.drop( [x for x in df_vacinas.columns if '_novas' in x or 'pessoas' in x], axis=1, inplace=True)
df_vacinas_detalhes.drop( [x for x in df_vacinas_detalhes.columns if '_novas' in x or 'pessoas' in x], axis=1, inplace=True)

df = df_vacinas.merge(df_vacinas_detalhes, how='left', on='data', suffixes=("_diario", "_semanal"))
df.data = df.data.apply(lambda x: f"{x[6:]}-{x[3:5]}-{x[0:2]}")
df.ffill(inplace=True)
#",".join(df.columns)

for i in ['', '1', '2']:
    df[f'doses{i}_diff'] = df[f'doses{i}_continente'] - df[f'doses{i}_diario']
cols = ['data'] + [ f'doses{i}_{j}' for i in ['', '1', '2'] for j in ['diff', 'continente', 'diario', 'semanal']]

df[ (df.data >= '2021-03-21') & (df.data <= '2021-03-29') ][ cols ]


In [ ]:

df['doses_diff'] = df['doses_diario'] - df['doses_continente']
df['doses_ilhas'] = df['doses_semanal'] - df['doses_continente']

df[['data', 'doses_diario', 'doses_continente', 'doses_diff', 'doses_madeira', 'doses_açores', 'doses_semanal', 'doses_ilhas']].tail(10)



In [ ]:
ilhas = 209905.0 + 188889.0
ilhas = 415064.0
diff = -6035.0
9284988.0 + diff + ilhas


In [ ]:
last = df_vacinas_detalhes[-1:]
last[[col for col in last.columns if 'doses' in col and not 'doses1' in col and not 'doses2' in col and not '9' in col and not 'ars' in col]]



In [ ]:
print('continente: ', last['doses_continente'])
print('soma ars: ', last['doses_arsnorte'] + last['doses_arscentro'] + last['doses_arslvt'] + last['doses_arsalentejo'] + last['doses_arsalgarve'])
print('diff continente: ', last['doses'] - last['doses_continente'])
print('ilhas: ', last['doses_madeira'] + last['doses_açores'])
print('falta: ', (last['doses'] - last['doses_continente']) - (last['doses_madeira'] + last['doses_açores']))



In [ ]:
# recebidas em falta 12 e 24
for k in ['recebidas', 'distribuidas']:
    x = data_vacinas_detalhes[k]
    #x = x[x > 0]
    #print(list(int(x) for x in list(x)))
    print(list(x))


In [ ]:
# primeira dose
df['doses1_7'] = df.doses1_diario.diff(7).div(7)
df['doses2_7'] = df.doses2_diario.diff(7).div(7)

df[['doses1_7', 'doses2_7']].plot(figsize=(15,4))
